In [7]:
import pandas as pd
import os,json

In [8]:
# need to clean up this block of code
csv_dir="logs"
program_file_dir="."
# get all program files that end with .py
program_files = [f for f in os.listdir(program_file_dir) if f.endswith(".py")]
program = program_files[0].split(".")[0]
same_program_csv_dir=os.path.join(csv_dir,program)
# samples = total number of runs for each program
samples = len(os.listdir(same_program_csv_dir))
op_to_func = {}

# loop through csv files in csv_dir
for program_ in program_files:
    program = program_.split(".")[0]
    same_program_csv_dir=os.path.join(csv_dir,program)
    common_funcs_per_op = set()
    c_func_to_count = {}
    for csv_file in os.listdir(same_program_csv_dir):
        df = pd.read_csv(os.path.join(same_program_csv_dir,csv_file))
        # merge two columns with "|" as separator
        df["full_func"] = df['Function'].astype(str) + "|" + df['Module'].astype(str)
        # loop through each function in the column 'full_func'
        for func in df["full_func"].values:
            # if function is not in the dictionary, add it
            if func not in c_func_to_count:
                c_func_to_count[func] = 1
            # else increment the count
            else:
                c_func_to_count[func] += 1

    for func in c_func_to_count:
        # i.e. present in more than half of the samples
        # if c_func_to_count[func] > samples//2:
        common_funcs_per_op.add(func)        

    op_to_func[program] = list(common_funcs_per_op)
    print(f"Common functions for {program} are {common_funcs_per_op}")

Common functions for BatchCollator are {'munmap|libc.so.6', 'gomp_team_end|libgomp.so.1', 'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::(anonymous namespace)::cpu_kernel_vec<(bool)1, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(at::vec::(anonymous namespace)::Vectorized<float>)#2}>(void, at::TensorIteratorBase&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}&&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{

In [9]:
op_to_func

{'BatchCollator': ['munmap|libc.so.6',
  'gomp_team_end|libgomp.so.1',
  'c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::(anonymous namespace)::cpu_kernel_vec<(bool)1, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(at::vec::(anonymous namespace)::Vectorized<float>)#2}>(void, at::TensorIteratorBase&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}&&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(at::vec::(

In [10]:

def find_common_cpp_func(op_to_func):
    # find functions found in other ops
    common_funcs_to_op = {}
    # loop through all programs
    for python_func in op_to_func.keys():
        # loop through all functions in the program
        for func in op_to_func[python_func]:
            # loop through all other programs
            for other_python_func in op_to_func.keys():
                # skip if the program is the same as the current program
                if other_python_func == python_func:
                    continue
                # if the function is found in other programs
                if func in op_to_func[other_python_func]:
                    # add the function to the set of common functions
                    if func not in common_funcs_to_op:
                        common_funcs_to_op[func] = [python_func,other_python_func]
                    elif python_func not in common_funcs_to_op[func] or other_python_func not in common_funcs_to_op[func]:
                        common_funcs_to_op[func].append(other_python_func)
    return common_funcs_to_op

In [11]:
common_funcs_to_op = find_common_cpp_func(op_to_func)
print("C/C++ functions that can be found in multiple python operations:")
for func in common_funcs_to_op.keys():
    print(f"\t{func}\n\t\t->\t{common_funcs_to_op[func]}")

C/C++ functions that can be found in multiple python operations:
	munmap|libc.so.6
		->	['BatchCollator', 'ToTensor', 'Normalize']
	gomp_team_end|libgomp.so.1
		->	['BatchCollator', 'ToTensor', 'Normalize']
	c10::function_ref<void (char**, long const*, long, long)>::callback_fn<at::TensorIteratorBase::loop_2d_from_1d<at::native::(anonymous namespace)::cpu_kernel_vec<(bool)1, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(at::vec::(anonymous namespace)::Vectorized<float>)#2}>(void, at::TensorIteratorBase&, at::native::(anonymous namespace)::copy_kernel(at::TensorIterator&, bool)::{lambda()#6}::operator()(void) const::{lambda()#8}::operator()(void) const::{lambda(float)#1}&&, at::native::(anonymous namespace)::

In [12]:
json_string = {'op_to_func':op_to_func,'common_funcs_to_op':common_funcs_to_op}
with open('mapping_funcs.json', 'w') as outfile:
    json.dump(json_string, outfile,indent=4)